# create_cmip6_globus_batch_files.ipynb
Create Globus batch files and scripts for ESGF CMIP6 data of interest.

B. Grandey, 2022.

In [1]:
! date

Tue Jan 18 17:16:16 +08 2022


In [2]:
import datetime
import pandas as pd
import pathlib
import pyesgf
from pyesgf.search import SearchConnection
import re

# Print versions of packages
for module in [pd, pyesgf, re]:
    try:
        print('{}.__version__ = {}'.format(module.__name__, module.__version__))
    except AttributeError:
        pass

pandas.__version__ = 1.3.5
pyesgf.__version__ = 0.3.0
re.__version__ = 2.2.1


## Base paths

In [3]:
# Base path in which to save batch files and scripts
out_base = pathlib.Path('cmip6_globus_batch_files/').resolve()
out_base.mkdir(exist_ok=True)  # create directory if it does not yet exist
# Directory in which to save local cache for search connection
cache_dir = pathlib.Path('cache/').resolve()
cache_dir.mkdir(exist_ok=True)

## Establish search connection

In [4]:
# Establish search connection
expire_after = datetime.timedelta(days=10)  # cache expiry
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search',
                        distrib=True,
                        cache='cache/pyesgf_cache',  # enable local cache
                        expire_after=expire_after)
conn

## Identify suitable sources (models) and members (ripf variants)
Do this by finding source-member pairs that fulfil the following requirements:
1. Monthly data are available for at least one of 'zostoga', 'zos', and 'tas' variables.
2. Data are available for at least one of 'ssp585', 'ssp370', 'ssp245', and 'ssp126' experiments.
3. Data are available for both 'historical' and 'piControl' experiments.
4. The member is an 'r1i1' variant (e.g. 'r1i1p1f1', 'r1i1p5f2').

In [5]:
%%time
# Create dictionary to hold available experiments (list) for each source-member pair (tuple)
source_member_experiment_dict = dict()
# Perform initial search for datasets matching first two requirements above
ctx1 = conn.new_context(project='CMIP6',
                        variable=['zostoga', 'zos', 'tas'],
                        frequency='mon',
                        experiment_id=['ssp585', 'ssp370', 'ssp245', 'ssp126'])
# Loop over available sources
sources = sorted(ctx1.facet_counts['source_id'].keys())
for source_id in sources:
    # Constrain search to source, to identify available members
    ctx2 = ctx1.constrain(source_id=source_id)
    # Find r1i1 members (requirement #4)
    members = sorted(ctx2.facet_counts['member_id'].keys())
    members = [m for m in members if bool(re.match('r1i1', m))]
    # Loop over members
    for member_id in members:
        # Search for available experiments for this source-member pair
        ctx3 = conn.new_context(project='CMIP6',
                                variable=['zostoga', 'zos', 'tas'],
                                frequency='mon',
                                source_id=source_id,
                                member_id=member_id)
        experiments = sorted(ctx3.facet_counts['experiment_id'].keys())
        # Limit to experiments of interest
        experiments = [e for e in experiments if e in ['piControl', 'historical',
                                                       'ssp585', 'ssp370', 'ssp245', 'ssp126']]
        # Are data available for both the historical and piControl experiments?
        if ('historical' in experiments) and ('piControl' in experiments):
            # Save to dictionary
            source_member_experiment_dict[(source_id, member_id)] = experiments
            # Print
            print(f'{source_id} {member_id}: {experiments}')
# Summarise number of source-member pairs identified
print(f'{len(source_member_experiment_dict)} source-member pairs identified.')

ACCESS-CM2 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
ACCESS-ESM1-5 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
AWI-CM-1-1-MR r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
BCC-CSM2-MR r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
BCC-ESM1 r1i1p1f1: ['historical', 'piControl', 'ssp370']
CAMS-CSM1-0 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CAS-ESM2-0 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CESM2 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CESM2-WACCM r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CIESM r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp585']
CMCC-CM2-SR5 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
CMCC-ESM2 r1i1p1f1: ['historical', 'piControl', 'ssp126', 'ssp

In [6]:
! date

Tue Jan 18 17:21:26 +08 2022
